# Get a list of tuples that has the name of all games and their url

In [31]:
import requests
from bs4 import BeautifulSoup

headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

base_url = 'https://www.metacritic.com/browse/games/score/metascore/all/pc'
res = requests.get(base_url, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")

last_page = int(soup.find('li', class_='page last_page').find('a').text)

game_data = []

for page in range(0, last_page):
    url = base_url + f'?page={page}'
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    
    game_links = soup.find_all("a", class_="title")
    
    for link in game_links:
        game_name = link.text.strip()
        game_url = 'https://www.metacritic.com' + link['href'] + '/user-reviews'
        game_data.append((game_name, game_url))

# Get the numger of comment pages each game has and include them into the tuples

In [23]:
game_urls = []
for game in game_data:
    name, url = game
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    pagination = soup.find('li', class_='page last_page')
    if pagination:
        page_number = int(pagination.find('a', class_='page_num').text)
    game_urls.append((name, url, page_number))

# Scrape the comments using the list of tuples: [('name', 'url', page_number)]

In [25]:
import time

def get_stats(game_urls):
    all_comments = []
    with requests.Session() as session:
        for game_name, url, num_pages in game_urls:
            comments = []
            for page_num in range(num_pages):
                page_url = f"{url}/user-reviews?page={page_num}"
                try:
                    response = session.get(page_url, headers=headers, allow_redirects=False)
                    if response.status_code == 200:
                        soup = BeautifulSoup(response.content, 'html.parser')
                        comment_tags = soup.find_all('div', {'class': 'review_body'})
                        comments += [comment.get_text().strip() for comment in comment_tags]
                    else:
                        print(f"Error: Could not retrieve page {page_num} for {game_name}.")
                except TypeError:
                    print(f"Error: {game_name} comments are NoneType and cannot be subscripted.")
                time.sleep(1)  # Add a delay between requests
            all_comments.append([game_name] + comments)
            print(f"Processed {game_name}, found {len(comments)} comments.")
    return all_comments

In [27]:
result = get_stats(game_urls)
print(result)

Processed Disco Elysium: The Final Cut, found 173 comments.
Processed Half-Life 2, found 2456 comments.
Processed Grand Theft Auto V, found 1439 comments.
Processed The Orange Box, found 273 comments.
Processed Half-Life, found 837 comments.
Processed BioShock, found 1019 comments.
Processed Baldur's Gate II: Shadows of Amn, found 257 comments.
Processed Persona 5 Royal, found 90 comments.
Processed Portal 2, found 2222 comments.
Processed The Elder Scrolls V: Skyrim, found 2688 comments.
Processed Mass Effect 2, found 784 comments.
Processed Grand Theft Auto: Vice City, found 302 comments.
Processed Sid Meier's Civilization II, found 64 comments.
Processed Quake, found 137 comments.
Processed BioShock Infinite, found 1882 comments.
Processed The Elder Scrolls IV: Oblivion, found 661 comments.
Processed Elden Ring, found 2038 comments.
Processed Grim Fandango, found 146 comments.
Processed Diablo, found 145 comments.
Processed Sid Meier's Civilization IV, found 214 comments.
Processed 

Processed Psychonauts 2, found 315 comments.
Processed Combat Mission: Barbarossa to Berlin, found 16 comments.
Processed Homeworld: Cataclysm, found 16 comments.
Processed Hi-Fi Rush, found 425 comments.
Processed The Walking Dead: Episode 5 - No Time Left, found 83 comments.
Processed Guild Wars, found 148 comments.
Processed Borderlands 2, found 715 comments.
Processed Max Payne, found 253 comments.
Processed Out of the Park Baseball 15, found 24 comments.
Processed Cuphead in the Delicious Last Course, found 44 comments.
Processed World in Conflict, found 88 comments.
Processed NASCAR Racing 4, found 22 comments.
Processed Duke Nukem 3D, found 70 comments.
Processed Far Cry, found 278 comments.
Processed Nuclear Throne, found 50 comments.
Processed Lone Echo, found 34 comments.
Processed FIFA Soccer 12, found 64 comments.
Processed XCOM: Enemy Unknown, found 623 comments.
Processed Mass Effect 3, found 2192 comments.
Processed Rise of Nations, found 117 comments.
Processed NASCAR R

Processed Out of the Park Baseball 4, found 8 comments.
Processed Monster Hunter: World - Iceborne, found 49 comments.
Processed F1 2019, found 22 comments.
Processed INSIDE, found 240 comments.
Processed Europa Universalis IV, found 298 comments.
Processed Persona 4 Golden, found 226 comments.
Processed Sunless Skies, found 25 comments.
Processed Total War: WARHAMMER II, found 121 comments.
Processed NHL 2004, found 30 comments.
Processed Zeus: Master of Olympus, found 18 comments.
Processed Hex: Shards of Fate, found 62 comments.
Processed World of Warcraft: Warlords of Draenor, found 419 comments.
Processed Warhammer 40,000: Dawn of War - Dark Crusade, found 90 comments.
Processed West of Loathing, found 42 comments.
Processed Iconoclasts, found 42 comments.
Processed Commandos 2: Men of Courage, found 56 comments.
Processed Forza Horizon 3, found 119 comments.
Processed Tales From The Borderlands: Episode 5 - The Vault of the Traveler, found 26 comments.
Processed Bastion, found 41

Processed Metro Exodus: Enhanced Edition, found 30 comments.
Processed Final Fantasy XV: Windows Edition, found 201 comments.
Processed Out of the Park Baseball 13, found 54 comments.
Processed Wizardry 8, found 34 comments.
Processed Aliens Versus Predator 2, found 44 comments.
Processed Operation Flashpoint: Cold War Crisis, found 48 comments.
Processed Tropico, found 29 comments.
Processed Giants: Citizen Kabuto, found 29 comments.
Processed NASCAR SimRacing, found 22 comments.
Processed The Lord of the Rings Online: Mines of Moria, found 40 comments.
Processed The Binding of Isaac: Afterbirth, found 38 comments.
Processed Amnesia: The Dark Descent, found 412 comments.
Processed Huntdown, found 26 comments.
Processed Chaos Reborn, found 40 comments.
Processed GTR: FIA GT Racing Game, found 34 comments.
Processed Control, found 408 comments.
Processed Football Manager 2011, found 27 comments.
Processed Dust: An Elysian Tail, found 129 comments.
Processed South Park: The Stick of Trut

Processed Age of Mythology: The Titans, found 47 comments.
Processed SimCity 4, found 83 comments.
Processed Microsoft Train Simulator, found 23 comments.
Processed Magic: The Gathering Arena, found 129 comments.
Processed Gordian Quest, found 7 comments.
Processed Lair of the Clockwork God, found 17 comments.
Processed Tales of Arise, found 82 comments.
Processed Fuga: Melodies of Steel, found 12 comments.
Processed Final Fantasy IX, found 30 comments.
Processed Black Mesa, found 242 comments.
Processed Rise of Nations: Rise of Legends, found 37 comments.
Processed TOCA Race Driver 3, found 24 comments.
Processed FTL: Faster Than Light, found 394 comments.
Processed Guacamelee! 2, found 26 comments.
Processed SOMA, found 257 comments.
Processed DEFCON: Everybody Dies, found 93 comments.
Processed Tom Clancy's The Division 2, found 567 comments.
Processed Tron 2.0, found 28 comments.
Processed NieR: Automata, found 338 comments.
Processed Brothers in Arms: Earned in Blood, found 37 com

...

Processed Prime World: Defenders, found 35 comments.
Processed Game of Thrones, found 98 comments.
Processed Diplomacy, found 16 comments.
Processed Peaky Blinders: Mastermind, found 14 comments.
Processed Gore: Ultimate Soldier, found 16 comments.
Processed Of Light and Darkness: The Prophecy, found 12 comments.
Processed Scrap Garden, found 14 comments.
Processed Formula Fusion, found 16 comments.
Processed Time Machine VR, found 18 comments.
Processed Vicious Circle, found 14 comments.
Processed Jack the Ripper, found 13 comments.
Processed Heroes of Might and Magic IV: Winds of War, found 15 comments.
Processed Vancouver 2010 - The Official Video Game of the Olympic Winter Games, found 13 comments.
Processed Undercover: Operation Wintersun, found 12 comments.
Processed Fade to Silence, found 28 comments.
Processed The Cameron Files: Pharaoh's Curse, found 12 comments.
Processed Tactical Ops: Assault on Terror, found 23 comments.
Processed Medieval Lords: Build, Defend, Expand, foun

Processed Homefront: The Revolution, found 83 comments.
Processed Moebius: Empire Rising, found 47 comments.
Processed CSI: Miami, found 13 comments.
Processed POSTAL Redux, found 23 comments.
Processed Gemini: Heroes Reborn, found 13 comments.
Processed Battle of Europe, found 6 comments.
Processed DOLMEN, found 20 comments.
Processed King Arthur: Fallen Champions, found 8 comments.
Processed True Crime: New York City, found 15 comments.
Processed Port Royale 3: Pirates and Merchants, found 24 comments.
Processed Colossal Cave, found 17 comments.
Processed The Political Machine 2012, found 12 comments.
Processed El Matador, found 17 comments.
Processed Night Watch, found 9 comments.
Processed Blind Fate: Edo no Yami, found 9 comments.
Processed SpellForce 2: Demons of the Past, found 10 comments.
Processed Real War: Rogue States, found 6 comments.
Processed Metal Gear Survive, found 65 comments.
Processed Duke Nukem Forever, found 479 comments.
Processed No Time to Explain, found 34 c

Processed RYL: Path of the Emperor, found 63 comments.
Processed Shade: Wrath of Angels, found 17 comments.
Processed Star Trek: D-A-C, found 16 comments.
Processed Trapped Dead, found 28 comments.
Processed Tryst, found 12 comments.
Processed America, found 9 comments.
Processed Art of Murder: FBI Confidential, found 8 comments.
Processed Legendary, found 31 comments.
Processed Don Bluth Presents: Dragon's Lair HD, found 7 comments.
Processed Animal Gods, found 6 comments.
Processed ShellShock: Nam '67, found 16 comments.
Processed Disney's Atlantis: The Lost Empire - Trial By Fire, found 8 comments.
Processed Reservoir Dogs, found 13 comments.
Processed RUNE II: Decapitation Edition, found 31 comments.
Processed Transformers: Rise of the Dark Spark, found 23 comments.
Processed Final Fantasy XIV Online, found 178 comments.
Processed YAIBA: NINJA GAIDEN Z, found 11 comments.
Processed Grand Theft Auto: The Trilogy - The Definitive Edition, found 2857 comments.
Processed Lara Croft Tom

Processed Wayward Manor, found 9 comments.
Processed Arcania: Fall of Setarrif, found 41 comments.
Processed Damnation, found 37 comments.
Processed Alien Blast: The Encounter, found 10 comments.
Processed Robot Arena, found 17 comments.
Processed 7554, found 23 comments.
Processed Timeline (2000), found 6 comments.
Processed DARK, found 60 comments.
Processed Seduce Me, found 13 comments.
Processed Brigade E5: New Jagged Union, found 21 comments.
Processed Mall Tycoon, found 10 comments.
Processed 4PM, found 9 comments.
Processed Delta Force: Xtreme 2, found 30 comments.
Processed Slain!, found 16 comments.
Processed Alekhine's Gun, found 14 comments.
Processed Evil Dead: Hail to the King, found 10 comments.
Processed The Chaos Engine (2013), found 10 comments.
Processed DRIV3R, found 35 comments.
Processed Made Man: Confessions of the Family Blood, found 13 comments.
Processed Psychotoxic, found 13 comments.
Processed ShellShock 2: Blood Trails, found 17 comments.
Processed Freedom: 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Convert the output list of lists into pandas dataframe

In [29]:
import pandas as pd

df = pd.DataFrame(columns=['Game', 'Comment'])

for inner_list in result:
    game = inner_list[0]
    comments = inner_list[1:]
    for comment in comments:
        df = df.append({'Game': game, 'Comment': comment}, ignore_index=True)

df

,Game,Comment
0,Disco Elysium: The Final Cut,"Still one of the best rpgs ever created, but n..."
1,Disco Elysium: The Final Cut,Communist propaganda garbage. Not worth playin...
2,Disco Elysium: The Final Cut,I'm reading through some of these negative rev...
3,Disco Elysium: The Final Cut,The game's strengths come from the intrigue of...
4,Disco Elysium: The Final Cut,One of the most unique and best written games ...
...,...,...
457150,Ride to Hell: Retribution,yknow what? this game is fun. story was bland....
457151,Ride to Hell: Retribution,This game is so hilariously bad its so good!! ...
457152,Ride to Hell: Retribution,What a prophetic title. Since the first announ...
457153,Ride to Hell: Retribution,The attempt to stick together some assets and ...


In [30]:
df.to_csv('Metacritic comments.csv', index=False)

# Convert comments into numeric values: total comments, #comments with keywords, %comments with keywords

In [31]:
import pandas as pd

df = pd.read_csv('Metacritic comments.csv')

Ncomments = df.groupby('Game').agg({'Comment': 'size'})

Ncomments

,Comment
Game,
.hack//G.U. Last Recode,25
007: NightFire,21
007: Quantum of Solace,23
0RBITALIS,6
10 Second Ninja,17
...,...
imprint-X,6
inMomentum,21
kill.switch,15


In [32]:
keywords = ['dizzy','dizziness','headache','tired','eyesore','fatigue','Dizzy','Dizziness','Tired', 'Headache','Eyesore','Fatigue']

# identify the rows that contain NaN values in the 'Comment' column
na_mask = df['Comment'].isna()

# filter the DataFrame to only include rows where the 'Comment' column contains any of the keywords and does not have any NaN values
filtered_df = df[df['Comment'].str.contains('|'.join(keywords)) & ~na_mask]

# group the filtered DataFrame by the 'Game' column and count the number of comments in each group
Ndiscomfort = filtered_df.groupby('Game').agg({'Comment': 'count'}).reset_index()

Ndiscomfort

,Game,Comment
0,A Game of Thrones: Genesis,1
1,A Hat in Time,1
2,A New Beginning - Final Cut,2
3,A Plague Tale: Innocence,2
4,A Plague Tale: Requiem,1
...,...,...
1159,Yu-Gi-Oh! Master Duel,1
1160,Zeno Clash,1
1161,Zeno Clash II,1
1162,Zombie Night Terror,2


In [33]:
merged_df = pd.merge(Ncomments, Ndiscomfort, on='Game', how='left')

merged_df = merged_df.rename(columns={'Comment_x': 'Ncomments'})

merged_df = merged_df.rename(columns={'Comment_y': 'Ndiscomfort'})

merged_df['Ndiscomfort'] = merged_df['Ndiscomfort'].fillna(0)

merged_df['Pdiscomfort'] = merged_df['Ndiscomfort']/merged_df['Ncomments']

merged_df

,Game,Ncomments,Ndiscomfort,Pdiscomfort
0,.hack//G.U. Last Recode,25,0.0,0.0
1,007: NightFire,21,0.0,0.0
2,007: Quantum of Solace,23,0.0,0.0
3,0RBITALIS,6,0.0,0.0
4,10 Second Ninja,17,0.0,0.0
...,...,...,...,...
5287,imprint-X,6,0.0,0.0
5288,inMomentum,21,0.0,0.0
5289,kill.switch,15,0.0,0.0
5290,nail'd,14,0.0,0.0


In [34]:
merged_df.to_csv('Cleaned Comments.csv', index=False)

# Scrape the APP ID for each games from SteamDB

In [1]:
!pip install selenium

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

s = Service("C:/Users/derek/OneDrive - Denison University/Downloads/chromedriver.exe")
chrome_options = Options()
chrome_options.binary_location = "C:/Users/derek/AppData/Local/Google/Chrome SxS/Application/chrome.exe"

driver = webdriver.Chrome(service=s, options=chrome_options)
driver.get("https://steamdb.info/charts/")

game_list = []

while True:
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    for game in soup.find_all('td'):
        a_tag = game.find('a')
        if a_tag:
            game_name = a_tag.text.strip()
            app_id = a_tag['href'].split('/')[2]
            game_list.append([game_name, app_id])

    next_button = driver.find_element(By.CSS_SELECTOR, 'a#table-apps_next')
    if 'disabled' in next_button.get_attribute('class'):
        break
    
    next_button.click()

driver.quit()

print(game_list)


[['', '730'], ['Counter-Strike: Global Offensive', '730'], ['', '570'], ['Dota 2', '570'], ['', '1172470'], ['Apex Legends', '1172470'], ['', '218'], ['Source SDK Base 2007', '218'], ['', '1085660'], ['Destiny 2', '1085660'], ['', '578080'], ['PUBG: BATTLEGROUNDS', '578080'], ['', '440'], ['Team Fortress 2', '440'], ['', '252490'], ['Rust', '252490'], ['', '1938090'], ['Call of Duty®: Modern Warfare® II | Warzone™ 2.0', '1938090'], ['', '1599340'], ['Lost Ark', '1599340'], ['', '2050650'], ['Resident Evil 4', '2050650'], ['', '271590'], ['Grand Theft Auto V', '271590'], ['', '218620'], ['PAYDAY 2', '218620'], ['', '236390'], ['War Thunder', '236390'], ['', '359550'], ["Tom Clancy's Rainbow Six Siege", '359550'], ['', '431960'], ['Wallpaper Engine', '431960'], ['', '1782210'], ['Crab Game', '1782210'], ['', '289070'], ["Sid Meier's Civilization VI", '289070'], ['', '346110'], ['ARK: Survival Evolved', '346110'], ['', '1811260'], ['EA SPORTS™ FIFA 23', '1811260'], ['', '1245620'], ['ELDE

In [10]:
Ngame_list = [game for game in game_list if '' not in game]
print(Ngame_list)

[['Counter-Strike: Global Offensive', '730'], ['Dota 2', '570'], ['Apex Legends', '1172470'], ['Source SDK Base 2007', '218'], ['Destiny 2', '1085660'], ['PUBG: BATTLEGROUNDS', '578080'], ['Team Fortress 2', '440'], ['Rust', '252490'], ['Call of Duty®: Modern Warfare® II | Warzone™ 2.0', '1938090'], ['Lost Ark', '1599340'], ['Resident Evil 4', '2050650'], ['Grand Theft Auto V', '271590'], ['PAYDAY 2', '218620'], ['War Thunder', '236390'], ["Tom Clancy's Rainbow Six Siege", '359550'], ['Wallpaper Engine', '431960'], ['Crab Game', '1782210'], ["Sid Meier's Civilization VI", '289070'], ['ARK: Survival Evolved', '346110'], ['EA SPORTS™ FIFA 23', '1811260'], ['ELDEN RING', '1245620'], ['Terraria', '105600'], ['Warframe', '230410'], ['Stardew Valley', '413150'], ['MIR4', '1623660'], ['The Sims™ 4', '1222670'], ['Football Manager 2023', '1904540'], ['Unturned', '304930'], ['Valheim', '892970'], ['DayZ', '221100'], ['Project Zomboid', '108600'], ['Hogwarts Legacy', '990080'], ['VRChat', '43810

# Convert the combination of game names and ids into Steam urls, and scrape the system requirement and release date for each game from Steam

In [108]:
url_list = [f"https://store.steampowered.com/app/{app_id}/{game_name.replace(' ', '_')}/" for game_name, app_id in Ngame_list]

print(url_list)

['https://store.steampowered.com/app/730/Counter-Strike:_Global_Offensive/', 'https://store.steampowered.com/app/570/Dota_2/', 'https://store.steampowered.com/app/1172470/Apex_Legends/', 'https://store.steampowered.com/app/218/Source_SDK_Base_2007/', 'https://store.steampowered.com/app/1085660/Destiny_2/', 'https://store.steampowered.com/app/578080/PUBG:_BATTLEGROUNDS/', 'https://store.steampowered.com/app/440/Team_Fortress_2/', 'https://store.steampowered.com/app/252490/Rust/', 'https://store.steampowered.com/app/1938090/Call_of_Duty®:_Modern_Warfare®_II_|_Warzone™_2.0/', 'https://store.steampowered.com/app/1599340/Lost_Ark/', 'https://store.steampowered.com/app/2050650/Resident_Evil_4/', 'https://store.steampowered.com/app/271590/Grand_Theft_Auto_V/', 'https://store.steampowered.com/app/218620/PAYDAY_2/', 'https://store.steampowered.com/app/236390/War_Thunder/', "https://store.steampowered.com/app/359550/Tom_Clancy's_Rainbow_Six_Siege/", 'https://store.steampowered.com/app/431960/Wal

In [340]:
exclude_list = ['https://store.steampowered.com/app/525480/.hack//G.U._Last_Recode/',
                'https://store.steampowered.com/app/841860/#Have_A_Sticker/',
                'https://store.steampowered.com/app/448350/Case_#8/']

new_list = [x for x in url_list if x not in exclude_list]

new_list

['https://store.steampowered.com/app/730/Counter-Strike:_Global_Offensive/',
 'https://store.steampowered.com/app/570/Dota_2/',
 'https://store.steampowered.com/app/1172470/Apex_Legends/',
 'https://store.steampowered.com/app/218/Source_SDK_Base_2007/',
 'https://store.steampowered.com/app/1085660/Destiny_2/',
 'https://store.steampowered.com/app/578080/PUBG:_BATTLEGROUNDS/',
 'https://store.steampowered.com/app/440/Team_Fortress_2/',
 'https://store.steampowered.com/app/252490/Rust/',
 'https://store.steampowered.com/app/1938090/Call_of_Duty®:_Modern_Warfare®_II_|_Warzone™_2.0/',
 'https://store.steampowered.com/app/1599340/Lost_Ark/',
 'https://store.steampowered.com/app/2050650/Resident_Evil_4/',
 'https://store.steampowered.com/app/271590/Grand_Theft_Auto_V/',
 'https://store.steampowered.com/app/218620/PAYDAY_2/',
 'https://store.steampowered.com/app/236390/War_Thunder/',
 "https://store.steampowered.com/app/359550/Tom_Clancy's_Rainbow_Six_Siege/",
 'https://store.steampowered.com

In [341]:
system_requirements = []

for url in new_list:
    response = requests.get(url)
    if response.url == 'https://store.steampowered.com/':
        print(f"Ignoring URL: {url}")
        continue
        
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('div', class_='apphub_AppName').get_text().strip()
    reqs = ""

    req_div = soup.find('div', class_='game_area_sys_req sysreq_content active')
    if req_div is not None:
        try:
            reqs = req_div.find('div', class_='game_area_sys_req_rightCol').find('ul', class_='bb_ul').get_text().strip()
        except AttributeError:
            ul = req_div.find('ul', class_='bb_ul')
            if ul is not None:
                lis = ul.find_all('li')
                for li in lis:
                    reqs += li.text.strip()
            else:
                req_ps = req_div.find_all('p')
                if len(req_ps) > 1:
                    reqs = req_ps[1].text
    system_requirements.append([title, reqs])

print(system_requirements)

Ignoring URL: https://store.steampowered.com/app/218/Source_SDK_Base_2007/
Ignoring URL: https://store.steampowered.com/app/480/Spacewar/
Ignoring URL: https://store.steampowered.com/app/212910/Call_of_Duty:_Black_Ops_II_-_Zombies/
Ignoring URL: https://store.steampowered.com/app/896660/Valheim_Dedicated_Server/
Ignoring URL: https://store.steampowered.com/app/624090/Football_Manager_2018/
Ignoring URL: https://store.steampowered.com/app/281130/Raven's_Cry/
Ignoring URL: https://store.steampowered.com/app/1371580/Myth_of_Empires/
Ignoring URL: https://store.steampowered.com/app/378120/Football_Manager_2016/
Ignoring URL: https://store.steampowered.com/app/2066750/Football_Manager_2023_Editor/
Ignoring URL: https://store.steampowered.com/app/581620/Bold_New_World/
Ignoring URL: https://store.steampowered.com/app/295270/Football_Manager_2015/
Ignoring URL: https://store.steampowered.com/app/231670/Football_Manager_2014/
Ignoring URL: https://store.steampowered.com/app/1190460/DEATH_STRAN

Ignoring URL: https://store.steampowered.com/app/223670/F1_2013/
Ignoring URL: https://store.steampowered.com/app/407250/Pro_Evolution_Soccer_2016_myClub/
Ignoring URL: https://store.steampowered.com/app/236830/Red_Orchestra_2:_Heroes_of_Stalingrad_-_Single_Player/
Ignoring URL: https://store.steampowered.com/app/49400/Magic:_The_Gathering_-_Duels_of_the_Planeswalkers/
Ignoring URL: https://store.steampowered.com/app/760620/Deiland/
Ignoring URL: https://store.steampowered.com/app/546450/North_Side/
Ignoring URL: https://store.steampowered.com/app/541180/PRO_EVOLUTION_SOCCER_2017_TRIAL_EDITION/
Ignoring URL: https://store.steampowered.com/app/216390/Fallen_Enchantress/
Ignoring URL: https://store.steampowered.com/app/220600/Football_Manager_2013_Editor/
Ignoring URL: https://store.steampowered.com/app/205930/Hitman:_Sniper_Challenge/
Ignoring URL: https://store.steampowered.com/app/200110/Nosgoth/
Ignoring URL: https://store.steampowered.com/app/449630/Rage_Parking_Simulator_2016/
Igno

Ignoring URL: https://store.steampowered.com/app/413510/The_Extinction/
Ignoring URL: https://store.steampowered.com/app/559200/Danny's_War/
Ignoring URL: https://store.steampowered.com/app/1993130/弈仙牌_Demo/
Ignoring URL: https://store.steampowered.com/app/603770/-_Arcane_RERaise_-/
Ignoring URL: https://store.steampowered.com/app/585790/Fergus_The_Fly/
Ignoring URL: https://store.steampowered.com/app/375760/Volstead/
Ignoring URL: https://store.steampowered.com/app/380790/Demon_Horde_Master/
Ignoring URL: https://store.steampowered.com/app/539700/SWEATER?_OK!/
Ignoring URL: https://store.steampowered.com/app/409440/SnakEscape/
Ignoring URL: https://store.steampowered.com/app/1904230/Exoprimal_Open_Beta_Test/
Ignoring URL: https://store.steampowered.com/app/407510/Devilian/
Ignoring URL: https://store.steampowered.com/app/582430/Jumping_Tank/
Ignoring URL: https://store.steampowered.com/app/555440/Deathgarden:_BLOODHARVEST/
Ignoring URL: https://store.steampowered.com/app/533620/STARUS

Ignoring URL: https://store.steampowered.com/app/636470/Tungulus/
Ignoring URL: https://store.steampowered.com/app/369200/Ghost_in_the_Shell:_Stand_Alone_Complex_-_First_Assault_Online/
Ignoring URL: https://store.steampowered.com/app/38830/CrimeCraft_GangWars/
Ignoring URL: https://store.steampowered.com/app/471230/Dead_by_Daylight:_BETA/
Ignoring URL: https://store.steampowered.com/app/456980/Parallels_Cross/
Ignoring URL: https://store.steampowered.com/app/591080/Undead_Hunter/
Ignoring URL: https://store.steampowered.com/app/952440/OVERKILL's_The_Walking_Dead_-_BETA/
Ignoring URL: https://store.steampowered.com/app/437890/AuroraRL/
Ignoring URL: https://store.steampowered.com/app/365310/Organ_Biker/
Ignoring URL: https://store.steampowered.com/app/386800/Street_Fighter_V_Beta/
Ignoring URL: https://store.steampowered.com/app/391560/Chicka_Wars_|_Chicken_Meat/
Ignoring URL: https://store.steampowered.com/app/431860/Color_Chemistry/
Ignoring URL: https://store.steampowered.com/app/24

Ignoring URL: https://store.steampowered.com/app/540310/GrottyScape/
Ignoring URL: https://store.steampowered.com/app/619520/Destruction_48/
Ignoring URL: https://store.steampowered.com/app/350710/Wizards'_Clash/
Ignoring URL: https://store.steampowered.com/app/205890/Chronicle:_RuneScape_Legends/
Ignoring URL: https://store.steampowered.com/app/391130/STEEL_RIVALS/
Ignoring URL: https://store.steampowered.com/app/523230/Super_Spring_Ninja/
Ignoring URL: https://store.steampowered.com/app/2020060/Underwater_Space_Demo/
Ignoring URL: https://store.steampowered.com/app/611780/Across_The_Moment/
Ignoring URL: https://store.steampowered.com/app/398070/Protoshift/
Ignoring URL: https://store.steampowered.com/app/1980640/Spider_Slingers_Demo/
Ignoring URL: https://store.steampowered.com/app/503220/Samurai_Wars/
Ignoring URL: https://store.steampowered.com/app/622530/Stoire/
Ignoring URL: https://store.steampowered.com/app/434780/The_Renegades_of_Orion_2.0/
Ignoring URL: https://store.steampo

Ignoring URL: https://store.steampowered.com/app/431990/Locked-in_syndrome/
Ignoring URL: https://store.steampowered.com/app/566990/Russian_Car_Driver/
Ignoring URL: https://store.steampowered.com/app/515310/Invention_2/
Ignoring URL: https://store.steampowered.com/app/575130/Danko_and_treasure_map/
Ignoring URL: https://store.steampowered.com/app/497350/Conclusion/
Ignoring URL: https://store.steampowered.com/app/574850/Hardnoid/
Ignoring URL: https://store.steampowered.com/app/628490/Zen_vs_Zombie/
Ignoring URL: https://store.steampowered.com/app/582480/Oldschool_tennis/
Ignoring URL: https://store.steampowered.com/app/616360/Crimson_Earth/
Ignoring URL: https://store.steampowered.com/app/209710/War_of_the_Immortals/
Ignoring URL: https://store.steampowered.com/app/1337120/Holy_SIsters_3D/
Ignoring URL: https://store.steampowered.com/app/625950/Pic_Guesser/
Ignoring URL: https://store.steampowered.com/app/526460/KickHim/
Ignoring URL: https://store.steampowered.com/app/612290/ASTA_On

Ignoring URL: https://store.steampowered.com/app/924720/For_Honor_-_Open_Test:_Marching_Fire/
Ignoring URL: https://store.steampowered.com/app/391300/Double_Cubes/
Ignoring URL: https://store.steampowered.com/app/448250/Nuked_Knight/
Ignoring URL: https://store.steampowered.com/app/596170/Magazine_Editor/
Ignoring URL: https://store.steampowered.com/app/506770/Foxhole_Pre-Alpha/
Ignoring URL: https://store.steampowered.com/app/205590/Legends_of_Pegasus/
Ignoring URL: https://store.steampowered.com/app/730590/Laser_League_Beta/
Ignoring URL: https://store.steampowered.com/app/496600/Partical_City_Guardians/
Ignoring URL: https://store.steampowered.com/app/574810/Fight_or_Die/
Ignoring URL: https://store.steampowered.com/app/629280/Run_Away/
Ignoring URL: https://store.steampowered.com/app/1753940/Starship_Troopers:_Terran_Command_Demo/
Ignoring URL: https://store.steampowered.com/app/857440/Hentai_Puzzle/
Ignoring URL: https://store.steampowered.com/app/382850/Piercing_Blow/
Ignoring UR

Ignoring URL: https://store.steampowered.com/app/1368460/Golden_Axed:_A_Cancelled_Prototype/
Ignoring URL: https://store.steampowered.com/app/457930/Starship:_Nova_Strike/
Ignoring URL: https://store.steampowered.com/app/527550/Platro/
Ignoring URL: https://store.steampowered.com/app/224320/Uncharted_Waters_Online/
Ignoring URL: https://store.steampowered.com/app/555430/On_The_Path/
Ignoring URL: https://store.steampowered.com/app/457440/Watch_This!/
Ignoring URL: https://store.steampowered.com/app/527460/Lepofrenia/
Ignoring URL: https://store.steampowered.com/app/593290/Moccasin/
Ignoring URL: https://store.steampowered.com/app/633370/Who's_your_Santa!?/
Ignoring URL: https://store.steampowered.com/app/218310/Frontline_Tactics/
Ignoring URL: https://store.steampowered.com/app/528210/Tracks_of_Triumph:_Summertime/
Ignoring URL: https://store.steampowered.com/app/415660/Tiger_Knight/
Ignoring URL: https://store.steampowered.com/app/575910/Trump_Simulator_2017/
Ignoring URL: https://sto

In [377]:
date = []
for url in new_list:
    response = requests.get(url)
    if response.url == 'https://store.steampowered.com/':
        print(f"Ignoring URL: {url}")
        continue
        
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('div', class_='apphub_AppName').get_text().strip()
    try:
        release_date = soup.find('div', class_='date').get_text().strip()
    except AttributeError:
        release_date = ''
    
    date.append([title, release_date])
    print(f"Scraped: {title}")
print(date)


Scraped: Counter-Strike: Global Offensive
Scraped: Dota 2
Scraped: Apex Legends™
Ignoring URL: https://store.steampowered.com/app/218/Source_SDK_Base_2007/
Scraped: Destiny 2
Scraped: PUBG: BATTLEGROUNDS
Scraped: Team Fortress 2
Scraped: Rust
Scraped: Call of Duty®: Modern Warfare® II
Scraped: Lost Ark
Scraped: Resident Evil 4
Scraped: Grand Theft Auto V
Scraped: PAYDAY 2
Scraped: War Thunder
Scraped: Tom Clancy's Rainbow Six® Siege
Scraped: Wallpaper Engine
Scraped: Crab Game
Scraped: Sid Meier’s Civilization® VI
Scraped: ARK: Survival Evolved
Scraped: EA SPORTS™ FIFA 23
Scraped: ELDEN RING
Scraped: Terraria
Scraped: Warframe
Scraped: Stardew Valley
Scraped: MIR4
Scraped: The Sims™ 4
Scraped: Football Manager 2023
Scraped: Unturned
Scraped: Valheim
Scraped: DayZ
Scraped: Project Zomboid
Scraped: Hogwarts Legacy
Scraped: VRChat
Scraped: FINAL FANTASY XIV Online
Scraped: Dead by Daylight
Scraped: Rocket League®
Scraped: Hearts of Iron IV
Scraped: Total War: WARHAMMER III
Scraped: Left 4

Scraped: MechWarrior 5: Mercenaries
Scraped: Against the Storm
Scraped: Terra Nil
Scraped: Nioh 2 – The Complete Edition
Scraped: Noita
Scraped: PlanetSide 2
Scraped: Football Manager 2020
Scraped: Dread Hunger
Scraped: BLEACH Brave Souls
Scraped: For The King
Scraped: The Sims™ 3
Scraped: Out There Somewhere
Scraped: Super Auto Pets
Scraped: Golf It!
Scraped: Muck
Scraped: World of Tanks
Scraped: Chivalry 2
Scraped: Timberborn
Scraped: Overcooked! 2
Scraped: Resident Evil Village
Scraped: World War Z
Scraped: Wo Long: Fallen Dynasty
Scraped: GrandChase
Scraped: Realm of the Mad God Exalt
Scraped: MX Bikes
Scraped: It Takes Two
Scraped: Starbound
Scraped: Cuphead
Scraped: Tom Clancy's Ghost Recon® Breakpoint
Scraped: FINAL FANTASY VII REMAKE INTERGRADE
Scraped: Conqueror's Blade
Scraped: Human: Fall Flat
Scraped: ROMANCE OF THE THREE KINGDOMS XIV
Scraped: PGA TOUR 2K23
Scraped: Middle-earth™: Shadow of War™
Scraped: Empyrion - Galactic Survival
Scraped: AdVenture Capitalist
Scraped: So

Scraped: Out of the Park Baseball 23
Scraped: Paragon: The Overprime
Scraped: Cosmoteer: Starship Architect & Commander
Scraped: One-armed cook
Scraped: Tribes of Midgard
Scraped: DEATH STRANDING DIRECTOR'S CUT
Scraped: DDraceNetwork
Scraped: Tactics Ogre: Reborn
Scraped: Tales of Arise
Scraped: Sniper Elite 4
Scraped: Rome: Total War™ - Collection
Scraped: Far Cry® 4
Scraped: Detroit: Become Human
Scraped: EARTH DEFENSE FORCE 5
Scraped: Realm Grinder
Scraped: Call of Duty®: Black Ops
Scraped: Castle Crashers®
Scraped: Call of Duty®: Black Ops Cold War
Scraped: Katana ZERO
Scraped: DOOM
Scraped: FINAL FANTASY XIV Online Free Trial
Scraped: SpaceBourne 2
Scraped: Left 4 Dead
Scraped: Wingspan
Scraped: Age of History II
Scraped: theHunter Classic
Scraped: Outer Wilds
Scraped: Moonbase 332
Scraped: Wildermyth
Scraped: Minion Masters
Scraped: Dragon Cliff
Scraped: Sleeping Dogs: Definitive Edition
Scraped: FPS Chess
Scraped: Half-Life
Scraped: Wargame: Red Dragon
Scraped: Space Station 14


Scraped: The Legend of Heroes: Trails to Azure
Scraped: Rogue Legacy 2
Scraped: Dishonored 2
Scraped: FINAL FANTASY® XIII
Scraped: Stranded Deep
Scraped: Automobilista 2
Scraped: Two Worlds II Castle Defense
Scraped: No More Room in Hell
Scraped: Age of Wonders: Planetfall
Scraped: TerraTech
Scraped: We Were Here
Scraped: Barony
Scraped: Knights of Honor II: Sovereign
Scraped: Tom Clancy’s The Division™
Scraped: Deadside
Scraped: Forts
Scraped: Sven Co-op
Scraped: Batman™: Arkham Origins
Scraped: NOBUNAGA'S AMBITION: Sphere of Influence - Ascension
Scraped: EverQuest
Scraped: BioShock Infinite
Scraped: Bloons TD Battles
Scraped: ENDER LILIES: Quietus of the Knights
Scraped: Session: Skate Sim
Scraped: Ori and the Blind Forest: Definitive Edition
Scraped: Terraforming Mars
Scraped: Max Payne 3
Scraped: The Riftbreaker
Scraped: Phantom Brigade
Scraped: Storybook Brawl
Scraped: NOBUNAGA'S AMBITION: Souzou with Power Up Kit
Scraped: The Outer Worlds
Scraped: DRAGON QUEST BUILDERS™ 2
Scrape

Scraped: Little Nightmares
Scraped: 河洛群俠傳 (Ho Tu Lo Shu ： The Books of Dragon)
Scraped: Fantasia Sango Mysteria
Scraped: Video Horror Society
Scraped: Overcooked! All You Can Eat
Scraped: A Little to the Left
Scraped: Euro Truck Simulator 2
Scraped: War Planet Online: Global Conquest
Scraped: Into the Breach
Scraped: Metro 2033 Redux
Scraped: Danganronpa 2: Goodbye Despair
Scraped: Dragon Age II: Ultimate Edition
Scraped: MELTY BLOOD: TYPE LUMINA
Scraped: Assassin's Creed® Syndicate
Scraped: ONE PIECE: PIRATE WARRIORS 4
Scraped: Call of War: World War 2
Scraped: Call of Duty®: Black Ops
Scraped: A Hat in Time
Scraped: Mafia II: Definitive Edition
Scraped: Everlasting Summer
Scraped: IL-2 Sturmovik: Battle of Stalingrad
Ignoring URL: https://store.steampowered.com/app/295270/Football_Manager_2015/
Scraped: GOD EATER 3
Scraped: The Great Ace Attorney Chronicles
Scraped: Smart Game Booster
Scraped: Phoenix Force
Scraped: House Party
Scraped: Planetary Annihilation: TITANS
Scraped: NAL Is 

...

Ignoring URL: https://store.steampowered.com/app/602220/SmashZombies/
Scraped: Galactic Junk League
Ignoring URL: https://store.steampowered.com/app/458900/Julai/
Scraped: Totally Unbalanced
Ignoring URL: https://store.steampowered.com/app/543780/Other_Tanks/
Ignoring URL: https://store.steampowered.com/app/521830/H.I.S.T.O.R.Y_T.O.R.C.H.K.A/
Scraped: Take Thy Throne
Scraped: Magicians & Looters
Ignoring URL: https://store.steampowered.com/app/537860/Ballistic_Protection/
Ignoring URL: https://store.steampowered.com/app/492130/Bye-Bye,_Wacky_Planet/
Ignoring URL: https://store.steampowered.com/app/397980/Invasion/
Scraped: Coma: Mortuary
Scraped: Worlds Adrift
Scraped: My Time at Sandrock
Ignoring URL: https://store.steampowered.com/app/2259670/Meet_Your_Maker_Demo_(Open_Beta)/
Scraped: Battleplan: American Civil War
Ignoring URL: https://store.steampowered.com/app/293560/TOME:_Immortal_Arena/
Scraped: Survival Is Not Enough
Scraped: The Ember Saga: A New Fire
Ignoring URL: https://sto

Ignoring URL: https://store.steampowered.com/app/461720/Tank_Blast/
Ignoring URL: https://store.steampowered.com/app/485220/The_Orb_Chambers/
Scraped: Green Elephant 2D
Scraped: Gladiator Trainer
Scraped: PRO EVOLUTION SOCCER 2018 LITE
Scraped: Wanda - A Beautiful Apocalypse
Scraped: YUME
Scraped: Platypus II
Ignoring URL: https://store.steampowered.com/app/543320/Alchemist/
Ignoring URL: https://store.steampowered.com/app/539350/Ball_of_Light/
Ignoring URL: https://store.steampowered.com/app/1869380/IXION_Demo/
Ignoring URL: https://store.steampowered.com/app/624140/Football_Manager_2018_Demo/
Ignoring URL: https://store.steampowered.com/app/449280/DinoOps/
Scraped: Fake Hostel
Scraped: Alone K.W.
Scraped: Super Killer Hornet: Resurrection
Scraped: Axis Football 2015
Ignoring URL: https://store.steampowered.com/app/434010/Wishmaster/
Scraped: Elementary My Dear Majesty!
Scraped: Going Medieval
Ignoring URL: https://store.steampowered.com/app/545280/The_face_of_hope:_Underground/
Ignor

Scraped: Steamy Sextet
Scraped: Fly in the House
Scraped: Super Blue Fighter
Ignoring URL: https://store.steampowered.com/app/629280/Run_Away/
Scraped: Mortos
Ignoring URL: https://store.steampowered.com/app/1753940/Starship_Troopers:_Terran_Command_Demo/
Scraped: Survival Kingdom
Ignoring URL: https://store.steampowered.com/app/857440/Hentai_Puzzle/
Scraped: Pipes!
Ignoring URL: https://store.steampowered.com/app/382850/Piercing_Blow/
Scraped: Football Manager 2021
Scraped: Everland (Stress Test)
Scraped: Snik
Scraped: CABAL Online
Ignoring URL: https://store.steampowered.com/app/463040/Experience/
Scraped: Street Warriors Online
Scraped: Within the blade
Scraped: Halloween Mysteries
Scraped: Farm Life: Natures Adventure
Scraped: Pid
Scraped: P-Walker's Simulation
Ignoring URL: https://store.steampowered.com/app/605520/Running_Sausage/
Scraped: Kidnapped
Scraped: crazy maze ~疯狂迷宫 ~ 狂った迷路 ~ Laberinto loco ~ Labyrinthe fou ~ Verrücktes Labyrinth
Ignoring URL: https://store.steampowered.

Scraped: Sniper Elite V2
Ignoring URL: https://store.steampowered.com/app/212180/Combat_Arms/
Scraped: Sullen
Scraped: Interloper
Scraped: Countless Rooms of Death
Scraped: Goggles - World of Vaporia
Ignoring URL: https://store.steampowered.com/app/1161150/龙魂传奇:76复古版MMO/
Ignoring URL: https://store.steampowered.com/app/1477610/Myth_of_Empires_Playtest/
Scraped: Grizzly Valley
Scraped: Winter Novel
Scraped: The Blue Flamingo
Scraped: Armikrog
Scraped: Deadlings: Rotten Edition
Scraped: Cooking Witch
Ignoring URL: https://store.steampowered.com/app/1815300/The_Wandering_Village_Demo/
Scraped: Empire of the Gods
Ignoring URL: https://store.steampowered.com/app/466480/Rage_Pig/
Scraped: Sid Meier's Civilization®: Beyond Earth™
Scraped: CRIMSON METAL Classic 1999
Ignoring URL: https://store.steampowered.com/app/1992090/活俠傳_Demo/
Scraped: King Lucas
Ignoring URL: https://store.steampowered.com/app/314700/Forsaken_Uprising/
Ignoring URL: https://store.steampowered.com/app/421650/Breach/
Scrap

Scraped: Crusaders of Light
Scraped: Monstro: Battle Tactics
Ignoring URL: https://store.steampowered.com/app/610390/Sugar_Box/
Scraped: Tribloos 2
Ignoring URL: https://store.steampowered.com/app/531490/Golden_Swords/
Scraped: Might & Magic Heroes Online
Scraped: 2 Ninjas 1 Cup
Ignoring URL: https://store.steampowered.com/app/491030/The_Last_Error/
Ignoring URL: https://store.steampowered.com/app/619730/Mad_Driver/
Scraped: ThrounnelVR
Ignoring URL: https://store.steampowered.com/app/524680/Escape_From_Nazi_Labs/
Ignoring URL: https://store.steampowered.com/app/686230/OutSplit/
Scraped: Leif's Adventure: Netherworld Hero
[['Counter-Strike: Global Offensive', 'Aug 21, 2012'], ['Dota 2', 'Jul 9, 2013'], ['Apex Legends™', 'Nov 4, 2020'], ['Destiny 2', 'Oct 1, 2019'], ['PUBG: BATTLEGROUNDS', 'Dec 21, 2017'], ['Team Fortress 2', 'Oct 10, 2007'], ['Rust', 'Feb 8, 2018'], ['Call of Duty®: Modern Warfare® II', 'Oct 27, 2022'], ['Lost Ark', 'Feb 11, 2022'], ['Resident Evil 4', 'Mar 23, 2023'],

In [393]:
import pandas as pd

df = pd.DataFrame(system_requirements, columns=['game_name', 'sys_req'])
datedf = pd.DataFrame(date, columns=['game_name', 'release_date'])

newdf = pd.merge(df, datedf, on='game_name')
newdf = newdf.drop_duplicates(subset=['game_name'])

newdf

,game_name,sys_req,release_date
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,"Aug 21, 2012"
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,"Jul 9, 2013"
2,Apex Legends™,Requires a 64-bit processor and operating syst...,"Nov 4, 2020"
3,Destiny 2,Requires a 64-bit processor and operating syst...,"Oct 1, 2019"
4,PUBG: BATTLEGROUNDS,Requires a 64-bit processor and operating syst...,"Dec 21, 2017"
...,...,...,...
5678,Tribloos 2,OS: Windows 7Processor: 2ghz Dual-CoreMemory: ...,"Jan 12, 2013"
5679,Might & Magic Heroes Online,Memory: 4 GB RAMGraphics: Hardware Accelerated...,"Nov 24, 2015"
5680,2 Ninjas 1 Cup,OS: Windows XPProcessor: Intel Pentium IV 2.8 ...,"Feb 3, 2017"
5681,ThrounnelVR,"OS: Windows 7, 8, 10Processor: 2.0Ghz dual-cor...","Sep 15, 2016"


# Create a new column that transform the format of game names in both datasets and merge them together

In [38]:
import re

def transform_game_name(name):
    return name.lower().replace(' ', '').replace(',', '').replace(':', '').replace('-', '').replace('.', '').replace("'", '')

def clean_name(name):
    name = re.sub(r'[^\w\s]', '', name)
    return name


newdf['name'] = newdf['game_name'].apply(lambda x: transform_game_name(x))
newdf['name'] = newdf['name'].apply(clean_name)

comments = pd.read_csv('Cleaned comments.csv')
comments['name'] = comments['Game'].apply(lambda x: transform_game_name(x))
comments['name'] = comments['name'].apply(clean_name)


In [395]:
merged_df = pd.merge(newdf, comments, on='name')

merged_df = merged_df.drop('Game', axis=1)

merged_df

,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,"Aug 21, 2012",counterstrikeglobaloffensive,1148,22.0,0.019164
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,"Jul 9, 2013",dota2,3829,17.0,0.004440
2,Apex Legends™,Requires a 64-bit processor and operating syst...,"Nov 4, 2020",apexlegends,550,3.0,0.005455
3,Destiny 2,Requires a 64-bit processor and operating syst...,"Oct 1, 2019",destiny2,559,4.0,0.007156
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,"Oct 10, 2007",teamfortress2,1635,16.0,0.009786
...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,"Jul 19, 2011",il2sturmovikcliffsofdover,76,0.0,0.000000
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,,dearesther,265,2.0,0.007547
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,"Apr 5, 2010",plainsight,18,1.0,0.055556
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,"Nov 4, 2009",shatteredhorizon,41,0.0,0.000000


In [ ]:
newdf.to_csv('raw_sys.csv', index=False)
comments.to_csv('Cleaned Comments.csv', index=False)
merged_df.to_csv('Finaldf.csv', index=False)

# Seperate the raw system requirement and extract each as a new column

In [204]:
import pandas as pd
import re

df = pd.read_csv('Finaldf.csv')

keys = ['OS:', 'Processor:', 'Memory:', 'Graphics:', 'DirectX:', 'Storage:', 'Network:']
req = list(df['sys_req'])
parsed = []
for e, i in enumerate(req):
    d = str(i)
    for z in keys:
        d = d.replace(z, '||'+z+'||')
    fields = d.split('||')

    these_keys = []
    these_vals = [] 
    for y in range(1, len(fields), 1):
        if y % 2 == 1:
            these_keys.append(fields[y])
        else:
            these_vals.append(fields[y])
    these_keys = [o.replace(':', '') for o in these_keys]

    if len(these_keys) == len(these_vals):
        if len(these_keys) == 0:
            result = {'no fields':e}
        else:
            result = dict(list(zip(these_keys, these_vals)))
    else:
        result = {'error': e}
    parsed.append(result)

reqdf = pd.DataFrame(parsed)
df = pd.concat([df, reqdf], axis=1)
df

,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,Graphics,DirectX,Storage,Network,no fields
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,Video card must be 256 MB or more and should ...,Version 9.0c,15 GB available space,NaN,NaN
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,"NVIDIA GeForce 8600/9600GT, ATI/AMD Radeon HD...",Version 11,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,"AMD Radeon™ R9 290, NVIDIA GeForce® GTX 970",Version 11,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,NVIDIA® GeForce® GTX 970 4GB or GTX 1060 6GB ...,NaN,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,NaN,Version 9.0c,15 GB available space,Broadband Internet connection,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,"DirectX® 10 compliant, 1GB Video Card (See su...",NaN,NaN,NaN,NaN
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,DirectX 9 compliant video card with Shader mod...,NaN,NaN,NaN,NaN
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,Shader model 2 supported graphics cardDirectX...,NaN,NaN,NaN,NaN
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,512MB NVIDIA GeForce GTX 260 / ATI Radeon HD ...,NaN,NaN,NaN,NaN


In [205]:
from collections import Counter 
all_os = []
for i in parsed:
    try:
        data = i['Processor']
        all_os.append(data)
    except:
        pass
Counter(all_os).most_common(100)

[(' 2 GHz', 6),
 (' Intel Core i5', 5),
 (' Dual Core Processor', 5),
 (' 2.0 Ghz', 4),
 (' Intel Core i7', 4),
 (' AMD/INTEL DUAL-CORE 2.5 GHZ', 4),
 (' Dual Core CPU', 4),
 (' Quad Core CPU', 4),
 (' Intel® Core™ i5-4570 3.20GHz', 3),
 (' 3GHz Quad Core', 3),
 (' AMD Ryzen™ 3 1200 / Intel® Core™ i5-6400', 3),
 (' Intel® Pentium® IV 2.4 GHz or AMD 3500+', 3),
 (' Quad-core Intel or AMD CPU ', 3),
 (' 3.2 GHz Quad Core Processor', 3),
 (' Core i5 or equivalent', 3),
 (' Intel Core 2 Duo E6400 / AMD 64 X2 4200+ ', 3),
 (' Intel Core i7-4790, AMD FX-8350', 3),
 (' Intel Core i5 4690 or AMD FX 8320', 3),
 (' Dual-core CPU with SSE3 (Intel® Pentium® D 3GHz / AMD Athlon™ 64 X2 4200) or better',
  3),
 (' 1.7 GHz Dual Core or Greater', 3),
 (' AMD/INTEL DUAL-CORE 2.4 GHZ', 3),
 (' AMD Ryzen 5 2600 / Intel Core i5-8600K', 3),
 (' Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 8750 processor or better', 2),
 (' Dual core from Intel or AMD at 2.8 GHz', 2),
 (' 2 Ghz', 2),
 (' Quad core CPU 3.0 GHz'

# Memory -> numeric

In [206]:
df['memory_num'] = df['Memory'].str.extract('(\d+)\s*(?:GB|MB)\s*', flags=re.IGNORECASE)
df['memory_num'] = pd.to_numeric(df['memory_num'], errors='coerce')
df.loc[df['memory_num'] > 100, 'memory_num'] /= 1024
df

,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,Graphics,DirectX,Storage,Network,no fields,memory_num
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,Video card must be 256 MB or more and should ...,Version 9.0c,15 GB available space,NaN,NaN,2.0
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,"NVIDIA GeForce 8600/9600GT, ATI/AMD Radeon HD...",Version 11,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN,4.0
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,"AMD Radeon™ R9 290, NVIDIA GeForce® GTX 970",Version 11,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN,8.0
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,NVIDIA® GeForce® GTX 970 4GB or GTX 1060 6GB ...,NaN,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN,8.0
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,NaN,Version 9.0c,15 GB available space,Broadband Internet connection,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,"DirectX® 10 compliant, 1GB Video Card (See su...",NaN,NaN,NaN,NaN,4.0
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,DirectX 9 compliant video card with Shader mod...,NaN,NaN,NaN,NaN,1.0
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,Shader model 2 supported graphics cardDirectX...,NaN,NaN,NaN,NaN,1.0
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,512MB NVIDIA GeForce GTX 260 / ATI Radeon HD ...,NaN,NaN,NaN,NaN,2.0


# OS -> numeric

In [208]:
df['OS_num'] = df['OS'].apply(lambda x: 0 
                              if pd.isna(x) 
                                  else (6 if '11' in x 
                                        else (5 if '10' in x 
                                              else (3 if '8' in x 
                                                    else (2 if '7' in x 
                                                          else (1 if 'XP' in x else 0))))))
df

,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,Graphics,DirectX,Storage,Network,no fields,memory_num,OS_num
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,Video card must be 256 MB or more and should ...,Version 9.0c,15 GB available space,NaN,NaN,2.0,2
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,"NVIDIA GeForce 8600/9600GT, ATI/AMD Radeon HD...",Version 11,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN,4.0,2
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,"AMD Radeon™ R9 290, NVIDIA GeForce® GTX 970",Version 11,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN,8.0,2
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,NVIDIA® GeForce® GTX 970 4GB or GTX 1060 6GB ...,NaN,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN,8.0,5
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,NaN,Version 9.0c,15 GB available space,Broadband Internet connection,NaN,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,"DirectX® 10 compliant, 1GB Video Card (See su...",NaN,NaN,NaN,NaN,4.0,2
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,DirectX 9 compliant video card with Shader mod...,NaN,NaN,NaN,NaN,1.0,1
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,Shader model 2 supported graphics cardDirectX...,NaN,NaN,NaN,NaN,1.0,1
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,512MB NVIDIA GeForce GTX 260 / ATI Radeon HD ...,NaN,NaN,NaN,NaN,2.0,2


In [209]:
for i, row in df.iterrows():
    if '64bit' in str(row['OS']):
        df.at[i, 'OS_num'] += 1
    elif '64-bit' in str(row['OS']):
        df.at[i, 'OS_num'] += 1
    elif '64 bit' in str(row['OS']):
        df.at[i, 'OS_num'] += 1
    elif '64 BIT' in str(row['OS']):
        df.at[i, 'OS_num'] += 1
    elif '64 Bit' in str(row['OS']):
        df.at[i, 'OS_num'] += 1
    elif '64-Bit' in str(row['OS']):
        df.at[i, 'OS_num'] += 1
    elif '64-bits' in str(row['OS']):
        df.at[i, 'OS_num'] += 1
    elif '64-BIT' in str(row['OS']):
        df.at[i, 'OS_num'] += 1
    elif '64Bit' in str(row['OS']):
        df.at[i, 'OS_num'] += 1

df

,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,Graphics,DirectX,Storage,Network,no fields,memory_num,OS_num
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,Video card must be 256 MB or more and should ...,Version 9.0c,15 GB available space,NaN,NaN,2.0,2
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,"NVIDIA GeForce 8600/9600GT, ATI/AMD Radeon HD...",Version 11,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN,4.0,2
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,"AMD Radeon™ R9 290, NVIDIA GeForce® GTX 970",Version 11,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN,8.0,3
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,NVIDIA® GeForce® GTX 970 4GB or GTX 1060 6GB ...,NaN,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN,8.0,6
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,NaN,Version 9.0c,15 GB available space,Broadband Internet connection,NaN,1.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,"DirectX® 10 compliant, 1GB Video Card (See su...",NaN,NaN,NaN,NaN,4.0,2
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,DirectX 9 compliant video card with Shader mod...,NaN,NaN,NaN,NaN,1.0,1
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,Shader model 2 supported graphics cardDirectX...,NaN,NaN,NaN,NaN,1.0,1
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,512MB NVIDIA GeForce GTX 260 / ATI Radeon HD ...,NaN,NaN,NaN,NaN,2.0,2


# Storage -> numeric

In [210]:
import numpy as np
import pandas as pd
import re

for index, row in df.iterrows():
    storage_value = re.findall(r'^\s(\d{1,4})', str(row['Storage']))
    if storage_value:
        df.at[index, 'storage_num'] = int(storage_value[0])

for i in range(len(df['storage_num'])):
    if df['storage_num'][i] > 180:
        df['storage_num'][i] = round(df['storage_num'][i] / 1024, 3)

df

<ipython-input-210-aa308a5553a7>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['storage_num'][i] = round(df['storage_num'][i] / 1024, 3)


,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,...,DirectX,Storage,Network,no fields,memory_num,OS_num,storage_num,graphics_simp,graphics_num,processor_simp
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,...,Version 9.0c,15 GB available space,NaN,NaN,2.0,2,15,NaN,5.0,DUO
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,...,Version 11,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN,4.0,2,60,GEFORCE 8600,5.2,DUAL
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,...,Version 11,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN,8.0,3,56,GTX 970,6.9,NaN
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,...,NaN,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN,8.0,6,105,GTX 970,6.9,I5-2400
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,...,Version 9.0c,15 GB available space,Broadband Internet connection,NaN,1.0,3,15,NaN,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,...,NaN,NaN,NaN,NaN,4.0,2,0,NaN,5.0,I5-2
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,...,NaN,NaN,NaN,NaN,1.0,1,0,NVIDIA 8800,5.2,QUAD
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,...,NaN,NaN,NaN,NaN,1.0,1,0,NaN,5.0,NaN
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,...,NaN,NaN,NaN,NaN,2.0,2,0,GTX 260,5.1,QUAD


# Graphics -> numeric (https://gpu.userbenchmark.com/)

In [224]:
import numpy as np

pattern1 = r'(GTX\s\d{3,4}\s?(?:Ti)?)'
pattern2 = r'(GeForce\s\d{3,4}\s?(?:Ti)?)'
pattern3 = r'(RTX \d{3,4}\s?(?:Ti)?)'
pattern4 = r'(GT \d{3,4}\s?(?:Ti)?)'
pattern5 = r'(NVIDIA \d{3,4}\s?(?:Ti)?)'
pattern6 = r'(GTX\d{3,4}\s?(?:Ti)?)'

match1 = df['Graphics'].str.extract(pattern1, flags=re.IGNORECASE, expand=False)
match2 = df['Graphics'].str.extract(pattern2, flags=re.IGNORECASE, expand=False)
match3 = df['Graphics'].str.extract(pattern3, flags=re.IGNORECASE, expand=False)
match4 = df['Graphics'].str.extract(pattern4, flags=re.IGNORECASE, expand=False)
match5 = df['Graphics'].str.extract(pattern5, flags=re.IGNORECASE, expand=False)
match6 = df['Graphics'].str.extract(pattern6, flags=re.IGNORECASE, expand=False)

conditions = [match1.notnull(), match2.notnull(), match3.notnull(), match4.notnull(), match5.notnull(), match6.notnull()]
choices = [match1, match2, match3, match4, match5, match6]
df['graphics_simp'] = np.select(conditions, choices, default=np.nan)

df['graphics_simp'] = df['graphics_simp'].str.upper().str.replace(r'GTX(\d+)', r'GTX \1')

df

<ipython-input-224-603ccd1176cd>:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df['graphics_simp'] = df['graphics_simp'].str.upper().str.replace(r'GTX(\d+)', r'GTX \1')


,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,...,Storage,Network,no fields,memory_num,OS_num,storage_num,graphics_simp,graphics_num,processor_simp,processor_num
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,...,15 GB available space,NaN,NaN,2.0,2,15,NaN,5.0,DUO,36.6
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,...,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN,4.0,2,60,GEFORCE 8600,5.2,DUAL,36.6
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,...,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN,8.0,3,56,GTX 970,6.9,NaN,27.7
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,...,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN,8.0,6,105,GTX 970,6.9,I5-2400,60.6
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,...,15 GB available space,Broadband Internet connection,NaN,1.0,3,15,NaN,5.0,NaN,27.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,...,NaN,NaN,NaN,4.0,2,0,NaN,5.0,I5-2,42.0
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,...,NaN,NaN,NaN,1.0,1,0,NVIDIA 8800,5.2,QUAD,45.5
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,...,NaN,NaN,NaN,1.0,1,0,NaN,5.0,NaN,27.7
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,...,NaN,NaN,NaN,2.0,2,0,GTX 260,5.1,QUAD,45.5


In [227]:
mapping = {'RTX 3070': 148, 'RTX 2080': 124, 'RTX 3060': 100, 'RTX 2070': 104, 'RTX 2060': 89.8, 'NVIDIA 9800': 7.51, 
           'NVIDIA 970': 49.1, 'NVIDIA 9400': 1.08, 'NVIDIA 8800': 7.11, 'NVIDIA 8600': 5.41, 'NVIDIA 8000': 5.41, 
           'NVIDIA 7900': 1.9, 'NVIDIA 7800': 1.31, 'NVIDIA 770': 34.1, 'NVIDIA 7600': 0.87, 'NVIDIA 6800': 0.16, 'NVIDIA 6600': 0.4, 
           'NVIDIA 500': 10.1, 'NVIDIA 460': 14.2, 'NVIDIA 320': 3.45, 'NVIDIA 260': 7.31, 'NVIDIA 1050 TI': 29.3, 'GTX 980': 59.3, 
           'GTX 970': 49.1, 'GTX 960': 32.9, 'GTX 950': 25.6, 'GTX 900': 33.4, 'GTX 780': 44.9, 'GTX 770': 34.1, 'GTX 760': 26, 
           'GTX 750': 14.7, 'GTX 740': 7.43, 'GTX 680': 31.6, 'GTX 670': 27.7, 'GTX 660': 19.9, 'GTX 650': 8.18, 'GTX 630': 3.62, 
           'GTX 600': 6.46, 'GTX 580': 22.8, 'GTX 570': 19.9, 'GTX 560': 13.8, 'GTX 555': 11.5, 'GTX 500': 7.4, 
           'GTX 480': 19.6, 'GTX 470': 16.2, 'GTX 460': 12.1, 'GTX 450': 7.38, 'GTX 400': 6.41, 'GTX 280': 8.47, 'GTX 275': 9.02, 
           'GTX 260': 7.31, 'GTX 2070': 104, 'GTX 1660': 68.3, 'GTX 1650': 42.8, 'GTX 1080': 102, 'GTX 1070': 79.7, 'GTX 1060': 56.1,
           'GTX 1050': 25.7, 'GTX 1000': 25.7, 'GT 760': 10.7, 'GT 750': 7.68, 'GT 730': 4.87, 'GT 650': 6.5, 'GT 640': 5.13, 
           'GT 610': 1.84, 'GT 512': 7.11, 'GT 450': 7.38, 'GT 430': 3.06, 'GT 240': 4.05, 'GT 220': 2.33, 'GT 1060': 46.7, 
           'GT 1030': 13.6, 'GEFORCE 9800': 6.48, 'GEFORCE 970': 30.7, 'GEFORCE 9600': 4.13, 'GEFORCE 960': 16.4, 'GEFORCE 9500': 2.04, 
           'GEFORCE 9400': 1.08, 'GEFORCE 9000': 0.55, 'GEFORCE 900': 0.96, 'GEFORCE 8800': 5.65, 
           'GEFORCE 8600': 2.25, 'GEFORCE 8000': 8.48, 'GEFORCE 7900': 1.9, 'GEFORCE 7800': 1.31, 'GEFORCE 780': 19.9, 
           'GEFORCE 770': 16, 'GEFORCE 7600': 0.87, 'GEFORCE 760': 10.7, 'GEFORCE 750': 7.68, 'GEFORCE 710': 3.48, 
           'GEFORCE 700': 6.43, 'GEFORCE 6800': 1.04, 'GEFORCE 6600': 0.4, 'GEFORCE 660': 7.26, 'GEFORCE 650': 6.5, 
           'GEFORCE 640': 5.13, 'GEFORCE 6100': 0.05, 'GEFORCE 610': 1.84, 'GEFORCE 600': 1.84, 'GEFORCE 560': 7.32, 
           'GEFORCE 500': 1.75, 'GEFORCE 480': 9.19, 'GEFORCE 470': 16.2, 'GEFORCE 460': 6.41, 'GEFORCE 450': 7.38, 
           'GEFORCE 400': 1.96, 'GEFORCE 320': 1.88, 'GEFORCE 260': 4.8, 'GEFORCE 240': 2.31, 'GEFORCE 2060': 89.8, 
           'GEFORCE 1080': 102, 'GEFORCE 1060': 56.1, 'GEFORCE 1050': 25.7, 'GTX 980 TI': 79, 'GTX 980TI': 79, 
           'GTX 780 TI': 52.9, 'GTX 750 TI': 17.2, 'GTX 750TI': 17.2, 'GTX 660 TI': 25.9, 'GTX 660TI': 25.9, 'GTX 650TI': 14.1, 
           'GTX 650 TI': 14.1, 'GTX 560TI': 16.3, 'GTX 560 TI': 16.3, 'GTX 550TI': 8.61, 'GTX 550 TI': 8.61, 'GTX 1660TI': 76, 
           'GTX 1660 TI': 76, 'GTX 1080 TI': 124, 'GTX 1070 TI': 93.7, 'GTX 1050 TI': 29.3, 'GTX 1050TI': 29.3, 
           'GEFORCE 750 TI': 17.2, 'GEFORCE 660 TI': 25.9, 'GEFORCE 560 TI': 16.3, 'GEFORCE 1080 TI': 124, 'GEFORCE 1050 TI': 29.3}

df['graphics_num'] = df['graphics_simp'].map(mapping).fillna(2)

df

,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,...,Storage,Network,no fields,memory_num,OS_num,storage_num,graphics_simp,graphics_num,processor_simp,processor_num
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,...,15 GB available space,NaN,NaN,2.0,2,15,NaN,2.00,DUO,36.6
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,...,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN,4.0,2,60,GEFORCE 8600,2.25,DUAL,36.6
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,...,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN,8.0,3,56,GTX 970,49.10,NaN,27.7
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,...,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN,8.0,6,105,GTX 970,2.00,I5-2400,60.6
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,...,15 GB available space,Broadband Internet connection,NaN,1.0,3,15,NaN,2.00,NaN,27.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,...,NaN,NaN,NaN,4.0,2,0,NaN,2.00,I5-2,42.0
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,...,NaN,NaN,NaN,1.0,1,0,NVIDIA 8800,7.11,QUAD,45.5
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,...,NaN,NaN,NaN,1.0,1,0,NaN,2.00,NaN,27.7
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,...,NaN,NaN,NaN,2.0,2,0,GTX 260,2.00,QUAD,45.5


# Processor -> numeric (https://cpu.userbenchmark.com/)

In [213]:
pattern1 = r'(i\d[-\s]?[0-9]+[CTKFS]?\b)'
pattern2 = r'(duo|dual|quad)'
pattern3 = r'(i\d\s*[- ]\s*\d+[KS]?\b)'


match1 = df['Processor'].str.extract(pattern1, flags=re.IGNORECASE, expand=False)
match2 = df['Processor'].str.extract(pattern2, flags=re.IGNORECASE, expand=False)
match3 = df['Processor'].str.extract(pattern3, flags=re.IGNORECASE, expand=False)

conditions = [match1.notnull(), match2.notnull(), match3.notnull()]
choices = [match1, match2, match3]
df['processor_simp'] = np.select(conditions, choices, default=np.nan)
df['processor_simp'] = df['processor_simp'].str.upper().str.replace(' ', '-')
df['processor_simp'] = df['processor_simp'].str.replace('--', '-')

df

,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,...,DirectX,Storage,Network,no fields,memory_num,OS_num,storage_num,graphics_simp,graphics_num,processor_simp
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,...,Version 9.0c,15 GB available space,NaN,NaN,2.0,2,15,NaN,5.0,DUO
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,...,Version 11,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN,4.0,2,60,GEFORCE 8600,5.2,DUAL
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,...,Version 11,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN,8.0,3,56,GTX 970,6.9,NaN
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,...,NaN,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN,8.0,6,105,GTX 970,6.9,I5-2400
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,...,Version 9.0c,15 GB available space,Broadband Internet connection,NaN,1.0,3,15,NaN,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,...,NaN,NaN,NaN,NaN,4.0,2,0,NaN,5.0,I5-2
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,...,NaN,NaN,NaN,NaN,1.0,1,0,NVIDIA 8800,5.2,QUAD
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,...,NaN,NaN,NaN,NaN,1.0,1,0,NaN,5.0,NaN
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,...,NaN,NaN,NaN,NaN,2.0,2,0,GTX 260,5.1,QUAD


In [215]:
mapping = {'I9-10900K': 99.6, 'I7-9700K': 96.9, 'I7-9700': 92, 'I7-950': 65.9, 'I7-920': 61.6, 'I7-8700K': 88.4, 
           'I7-8700': 87, 'I7-8600K': 90.7, 'I7-8600': 88.2, 'I7-7700K': 84, 'I7-7700': 76.2, 'I7-6700K': 80.8, 
           'I7-6700': 72.3, 'I7-5930K': 79.8, 'I7-5820K': 76.8, 'I7-5775C': 84.9, 'I7-4790K': 78.9, 'I7-4790': 71.4, 
           'I7-4770S': 71.2, 'I7-4770K': 73.6, 'I7-4770': 70.5, 'I7-3970': 77.9, 'I7-3930K': 74.4, 'I7-3820': 69.5, 
           'I7--3770K': 72.4, 'I7-3770K': 72.4, 'I7-3770': 69, 'I7-3700': 60.8, 'I7-3': 60.8, 'I7-2700K': 73, 'I7-2600K': 70.6, 
           'I7-2600': 67.1, 'I7-2': 67.1, 'I7-11700': 98.1, 'I7-10700K': 96.5, 'I5-9600K': 93.2, 'I5-9600': 92.3, 'I5-9500F': 87, 
           'I5-9500': 87, 'I5-9400F': 84.8, 'I5-9400': 86.5, 'I5-8600K': 90.7, 'I5-8600': 88.2, 'I5-8500': 82.1, 'I5-8400': 81.8, 
           'I5-7600K': 75.8, 'I5-7600': 72, 'I5-760': 63.6, 'I5-7500': 66.8, 'I5-750': 61.5, 'I5-7400S': 65.3, 'I5-7400': 63.5, 
           'I5--6600K': 72.4, 'I5-6600K': 72.4, 'I5-6600': 67.6, 'I5-6500': 63.6, 'I5-650': 46.3, 'I5-6400': 60.8, 'I5-5675C': 81.8,
           'I5-4690K': 69.6, 'I5-4690': 67.1, 'I5-4670K': 68, 'I5-4670': 66.5, 'I5-4590': 63.5, 'I5-4570T': 57.7, 'I5-4570': 62.4, 
           'I5-4460': 62.1, 'I5-4440': 61.3, 'I5-4430': 65.2, 'I5-4000': 65.2, 'I5-3770': 49, 'I5-3570K': 67.6, 'I5-3570': 65.1, 
           'I5-350K': 60.8, 'I5-3470': 63.1, 'I5-3300': 60.8, 'I5-3000': 57.6, 'I5-3': 57.6, 'I5-2520': 47.6, 
           'I5-2500K': 55.2, 'I5-2500': 62.9, 'I5-2400S': 60.4, 'I5-2400': 60.6, 'I5-2320': 63.9, 
           'I5-2300': 64, 'I5-2100': 41.9, 'I5-2': 42, 'I5-11600K': 99.2, 'I5-11400': 94.2, 
           'I5-10505': 70.2, 'I3-8350K': 81.7, 'I3-8100': 70.6, 'I3-6300': 66.7, 'I3-6100T': 57.6, 
           'I3-6100': 55.4, 'I3-560': 51.6, 'I3-550': 47, 'I3-530': 45.6, 'I3-4340': 64.5, 
           'I3-4330': 61.9, 'I3-4170': 56.9, 'I3-4160': 55.1, 'I3-4130': 52.3, 'I3-3240': 55.1, 
           'I3-3': 51.4, 'I3-2120': 50.7, 'I3-2100T': 52.6, 'I3-2100': 50.9, 'I3-2': 50.9, 
           'QUAD': 45.5, 'DUO': 36.6, 'DUAL': 36.6}

df['processor_num'] = df['processor_simp'].map(mapping).fillna(27.7)

df

,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,...,Storage,Network,no fields,memory_num,OS_num,storage_num,graphics_simp,graphics_num,processor_simp,processor_num
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,...,15 GB available space,NaN,NaN,2.0,2,15,NaN,5.0,DUO,36.6
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,...,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN,4.0,2,60,GEFORCE 8600,5.2,DUAL,36.6
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,...,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN,8.0,3,56,GTX 970,6.9,NaN,27.7
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,...,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN,8.0,6,105,GTX 970,6.9,I5-2400,60.6
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,...,15 GB available space,Broadband Internet connection,NaN,1.0,3,15,NaN,5.0,NaN,27.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,...,NaN,NaN,NaN,4.0,2,0,NaN,5.0,I5-2,42.0
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,...,NaN,NaN,NaN,1.0,1,0,NVIDIA 8800,5.2,QUAD,45.5
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,...,NaN,NaN,NaN,1.0,1,0,NaN,5.0,NaN,27.7
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,...,NaN,NaN,NaN,2.0,2,0,GTX 260,5.1,QUAD,45.5


In [228]:
df.to_csv('Finaldf.csv', index=False)